## Train Model

### Configure Spark for Your Notebook
* This examples uses the local Spark Master `--master local[1]`
* In production, you would use the PipelineIO Spark Master `--master spark://apachespark-master-2-1-0:7077`

In [ ]:
import os

master = '--master local[1]'
#master = '--master spark://apachespark-master-2-1-0:7077'
conf = '--conf spark.cores.max=1 --conf spark.executor.memory=512m'
packages = '--packages com.amazonaws:aws-java-sdk:1.7.4,org.apache.hadoop:hadoop-aws:2.7.1'
jars = '--jars /root/lib/jpmml-sparkml-package-1.0-SNAPSHOT.jar'
py_files = '--py-files /root/lib/jpmml.py'

os.environ['PYSPARK_SUBMIT_ARGS'] = master \
  + ' ' + conf \
  + ' ' + packages \
  + ' ' + jars \
  + ' ' + py_files \
  + ' ' + 'pyspark-shell'

print(os.environ['PYSPARK_SUBMIT_ARGS'])

### Import Spark Libraries

In [ ]:
from pyspark.ml.linalg import Vectors
from pyspark.ml.feature import VectorAssembler, StandardScaler
from pyspark.ml.feature import OneHotEncoder, StringIndexer
from pyspark.ml import Pipeline, PipelineModel
from pyspark.ml.regression import LinearRegression

### Configure Spark for Your Notebook
* You may need to Reconnect and/or Restart the Kernel to pick up changes.
* This examples uses the local Spark Master `--master local[1]`
* In production, you would use the PipelineIO Spark Master `--master spark://apachespark-master-2-1-0:7077`zzaa

### Create Spark Session
This may take a minute or two.  Please be patient.

### Read Data from Public S3 Bucket
* AWS credentials are not needed.
* We're asking Spark to infer the schema
* The data has a header
* Using `bzip2` because it's a splittable compression file format

In [ ]:
df = spark_session.read.format("csv") \
    .option("inferSchema", "true") \
    .option("header", "true") \
    .load("s3a://datapalooza/airbnb/airbnb.csv.bz2")

print(df.head())

In [ ]:
print(df.count())

### Clean, Filter, and Summarize the Data

In [3]:
df_filtered = df.filter("price >= 50 AND price <= 750 AND bathrooms > 0.0 AND bedrooms is not null")

df_filtered.registerTempTable("df_filtered")

df_final = spark_session.sql("""
    select
        id,
        city,
        case when state in('NY', 'CA', 'London', 'Berlin', 'TX' ,'IL', 'OR', 'DC', 'WA')
            then state
            else 'Other'
        end as state,
        space,
        cast(price as double) as price,
        cast(bathrooms as double) as bathrooms,
        cast(bedrooms as double) as bedrooms,
        room_type,
        host_is_super_host,
        cancellation_policy,
        cast(case when security_deposit is null
            then 0.0
            else security_deposit
        end as double) as security_deposit,
        price_per_bedroom,
        cast(case when number_of_reviews is null
            then 0.0
            else number_of_reviews
        end as double) as number_of_reviews,
        cast(case when extra_people is null
            then 0.0
            else extra_people
        end as double) as extra_people,
        instant_bookable,
        cast(case when cleaning_fee is null
            then 0.0
            else cleaning_fee
        end as double) as cleaning_fee,
        cast(case when review_scores_rating is null
            then 80.0
            else review_scores_rating
        end as double) as review_scores_rating,
        cast(case when square_feet is not null and square_feet > 100
            then square_feet
            when (square_feet is null or square_feet <=100) and (bedrooms is null or bedrooms = 0)
            then 350.0
            else 380 * bedrooms
        end as double) as square_feet
    from df_filtered
""").persist()

df_final.registerTempTable("df_final")

df_final.select("square_feet", "price", "bedrooms", "bathrooms", "cleaning_fee").describe().show()

NameError: name 'df' is not defined

In [4]:
print(df_final.count())

NameError: name 'df_final' is not defined

## Export the Spark ML Pipeline

In [ ]:
from jpmml import toPMMLBytes

model = toPMMLBytes(spark_session, df, pipeline_model)
with open('model.spark', 'wb') as fh:
    fh.write(model)